In [1]:
config='EU36'
case='MPC001'
freq='1d'

year='2012'


In [2]:
# Parameters
config = "EU36"
case = "MPC002"


In [3]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()
c

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:40900 Dashboard: http://127.0.0.1:34826/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [4]:
## path for mdules

import sys
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/xscale")
import xscale

sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/Vorticity_Variance/")
import GriddedData

## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

%matplotlib inline

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [5]:
if case == 'MPC001':
    if config == 'EU36':
        gridfile='/scratch/cnt0024/hmg2840/colombo/'+config+'/'+config+'-I/coordinates_'+config+'.nc'
        maskfile='/scratch/cnt0024/hmg2840/colombo/'+config+'/'+config+'-I/'+config+'.L75_mesh_mask.nc'
    if config == 'GS36':
        gridfile='/scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I/GS36_coordinates.nc'
        maskfile='/scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I/GS36.L75-MAA007_mesh_mask.nc'
if case == 'MPC002':
    if config == 'EU36':
        gridfile='/scratch/cnt0024/hmg2840/colombo/EU36/EU36-I.L150_1/coordinates_EU36.nc'
        maskfile='/scratch/cnt0024/hmg2840/colombo/EU36/EU36-I.L150_1/EU36.L150_1_mesh_mask.nc'
    if config == 'GS36':
        gridfile='/scratch/cnt0024/hmg2840/colombo/GS36/GS36-I.L150_1/GS36_coordinates.nc'
        maskfile='/scratch/cnt0024/hmg2840/colombo/GS36/GS36-I.L150_1/GS36.L150_1_mesh_mask.nc'
if case == 'MPC003':
    if config == 'EU36':
        gridfile='/scratch/cnt0024/hmg2840/colombo/EU36/EU36-I.L150_2/coordinates_EU36.nc'
        maskfile='/scratch/cnt0024/hmg2840/colombo/EU36/EU36-I.L150_2/EU36.L150_2_mesh_mask.nc'
    if config == 'GS36':
        gridfile='/scratch/cnt0024/hmg2840/colombo/GS36/GS36-I.L150_2/GS36_coordinates.nc'
        maskfile='/scratch/cnt0024/hmg2840/colombo/GS36/GS36-I.L150_2/GS36.L150_2_mesh_mask.nc'
 

In [6]:
## Dataset

dirs="/scratch/cnt0024/hmg2840/colombo/GS36/GS36-MPC001-S/1d/2010/"
dirin="/scratch/cnt0024/hmg2840/colombo/GS36/GS36-I-Aur/"


In [7]:
grid=xr.open_dataset(gridfile)
navlat= grid['nav_lat']
navlon= grid['nav_lon']


In [8]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=12, dim=['x', 'y'], n=[18, 18])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [9]:
def plot_fine_scale_variance(var,loncrs,latcrs,lon,lat,hpvarm,month,m,config,case):
    ''' map of the averaged fine scale variance
    '''
    if config == 'EU36':
        fig, ax = plt.subplots(1,1,figsize=(10,15))
    if config == 'GS36':
        fig, ax = plt.subplots(1,1,figsize=(15,10))
    ax = plt.subplot(111,projection=ccrs.PlateCarree(central_longitude=0))
    ax.autoscale(tight=True)
    cont=np.isnan(hpvarm)
    gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                      alpha=0.5)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    
    ax.tick_params('both',labelsize=25)

    pcolor = ax.pcolormesh(loncrs,latcrs,ma.masked_invalid(var),cmap=seq_cmap,vmin=0,vmax=0.1,alpha=1)
    ax.contour(lon,lat,cont,alpha=0.5,linewidth=0.000001,antialiased=True,colors='black')
    cbar = plt.colorbar(pcolor,orientation='horizontal',pad=0.1)
    cbar.ax.tick_params(labelsize=35)
    cbar.ax.tick_params(labelsize=25)
    cbar.set_label('Small scales surface vorticity variance in '+month+' for '+config+'-'+case,fontsize=15)
    plt.savefig('Plots/'+config+'-'+case+'_y'+year+'m'+m+'_fine_scale_variance_vorticity.png')
  

In [10]:
def plot_vorticity_variance(month):
    date_list=[]
    if month == '03':
        dayf=31
    if month == '09':
        dayf=30
    for day in np.arange(1,dayf+1):
        if day < 10:
            date='y'+str(year)+'m'+str(month)+'d0'+str(day)
        else:
            date='y'+str(year)+'m'+str(month)+'d'+str(day)
        date_list.append(date)
    datadir='/scratch/cnt0024/hmg2840/colombo/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'
    for date in date_list:
        fileU=datadir+'*/'+str(config)+'-'+str(case)+'_'+str(date)+'.'+str(freq)+'_gridU.nc'
        fileV=datadir+'*/'+str(config)+'-'+str(case)+'_'+str(date)+'.'+str(freq)+'_gridV.nc'
        filecurl='/scratch/cnt0024/hmg2840/albert7a/tmp/'+str(config)+'-'+str(case)+'_'+str(date)+'.'+str(freq)+'_socurloverf.nc'
        if not os.path.exists(filecurl):
            print('Computing '+str(filecurl))
            !ln -sf $maskfile mesh_hgr.nc
            !ln -sf $maskfile mesh_zgr.nc
            !cdfcurl -u $fileU vozocrtx -v $fileV vomecrty -l 1 -surf -overf -o $filecurl
            !rm mesh*
    filescurl='/scratch/cnt0024/hmg2840/albert7a/tmp/'+str(config)+'-'+str(case)+'_y'+str(year)+'m'+str(month)+'d??.'+str(freq)+'_socurloverf.nc'
    dscurl=xr.open_mfdataset(filescurl,chunks={'time_counter':1,'y':700,'x':1000})
    curl=dscurl.socurloverf
    curl_SS=filt(curl)
    curl_LS=curl-curl_SS
    hpcurl=curl_SS
    hpcurl2 = hpcurl ** 2
    hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=36)
    hpcurl2mc = crs.return_ravel(np.asarray(hpcurl2m))
    hpcurl2mcm = np.mean(hpcurl2mc,axis=-3)
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)
    if month == '03':
        plot_fine_scale_variance(hpcurl2mcm,loncrsm, latcrsm,navlon,navlat,hpcurl2m,'March',month,config,case)
    if month == '09':
        plot_fine_scale_variance(hpcurl2mcm,loncrsm, latcrsm,navlon,navlat,hpcurl2m,'September',month,config,case)
        

In [11]:
month_list=['03','09']

for month in month_list:
    if not os.path.exists('Plots/'+config+'-'+case+'_y'+year+'m'+month+'_fine_scale_variance_vorticity.png'):
        plot_vorticity_variance(month)